<h1 style="color:crimson">Tarea</h1>

* Entrenar un modelo Lasso con GridSearchCV usando un pipeline de polinomios de segundo grado
* Graficar coeficientes "significativos"
* Revisar cuantos parámetros son mayores a 1e-2

In [ ]:
## Hint: usando parámetros dentro de un pipeline
# para usar dentro de GridSearchCV
pipe = Pipeline([
    "paso1": PolynomialFeatures(),
    "model": LinearRegression()
])

params = {
    "model__normalize": [True, False]
}

In [65]:
#Se importan todas las librerías necesarias
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import plotly.express as px
from numpy.random import uniform
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate

In [66]:
#se buscan los datos y se muestran
ipc_path = "ipc.xlsx" 
market = pd.read_excel(ipc_path, index_col=0)
market.head()

,KIMBERA MF Equity,GAPB MF Equity,CEMEXCPO MF Equity,ASURB MF Equity,GFNORTEO MF Equity,KOFUBL MF Equity,AC* MF Equity,LABB MF Equity,FEMSAUBD MF Equity,OMAB MF Equity,...,MEGACPO MF Equity,GCC* MF Equity,BIMBOA MF Equity,GMEXICOB MF Equity,GCARSOA1 MF Equity,TLEVICPO MF Equity,ALFAA MF Equity,BSMXB MF Equity,PINFRA* MF Equity,MEXBOL INDEX
DATE,,,,,,,,,,,,,,,,,,,,,
2016-01-01,40.37,152.05,8.72,243.89,94.99,123.90,104.68,13.85,161.63,83.61,...,64.19,44.81,45.95,36.79,70.98,94.34,34.10,30.25,202.73,42977.50
2016-01-04,39.16,148.57,8.43,238.29,91.79,120.29,102.09,14.41,157.62,81.77,...,64.45,43.51,45.02,35.71,69.83,92.62,33.74,29.46,202.93,42113.70
2016-01-05,38.89,147.93,8.31,233.03,91.72,118.82,101.12,13.77,157.80,81.58,...,63.16,43.51,45.77,36.45,70.70,91.89,33.99,29.72,203.56,42041.68
2016-01-06,39.13,146.99,8.29,231.97,91.63,120.32,102.50,13.65,156.37,82.03,...,63.27,43.51,45.97,35.57,70.47,90.41,33.65,29.67,199.80,41691.19
2016-01-07,39.16,141.32,7.64,230.71,89.54,120.20,101.16,13.25,155.14,80.94,...,62.78,42.76,46.18,35.01,68.15,90.23,31.46,28.50,196.75,40661.57


In [67]:
Rmkt = market.pct_change().dropna()

Xall = Rmkt.drop("MEXBOL INDEX", axis=1)
Xall = Xall.join(Rmkt["MEXBOL INDEX"].shift(1)).dropna()

Xtrain = Xall.drop("MEXBOL INDEX", axis=1)
ytrain = Xall["MEXBOL INDEX"]

In [68]:
# Se defdinen los periodos del entrenamiento y la prueba
test_period = "2019-12"
Xtest, ytest = Xtrain[test_period:], ytrain[test_period:]
Xtrain, ytrain = Xtrain[:"2019-11":], ytrain[:"2019-11"]

In [69]:

ytrain.shape, ytest.shape

((646,), (25,))

In [70]:
#se emplea una alpha uniforme que sirva para entrenar el modelo
alpha_list = uniform(0.0001, 0.000001, 300)

In [71]:
# Se crea el Pipeline del modelo
pipe = Pipeline([
    ("paso1", PolynomialFeatures(degree=2)),
    ("model", Lasso())
])

# Se determinan los parámetros a usar para el entrenamiento
params = {
    "model__normalize": [True, False],
    "model__alpha": alpha_list,
    "model__fit_intercept": [True, False]

}

In [72]:
# Se determina el SearcCV con lo anteriormente definido
SEARCH = GridSearchCV(estimator = pipe,
                    param_grid = dict(params),
                    cv = 5)
gs=SEARCH.fit(Xtrain, ytrain)

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small a

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.

C:\Users\danie\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning:

Objective did not converge. You might want to increase the number of iterations. Fitting data with very small a

In [74]:
gs.best_params_

{'model__alpha': 8.641139163738937e-06,
 'model__fit_intercept': True,
 'model__normalize': False}

In [75]:
gs.predict(Xtest)

array([-1.64426220e-03, -1.03632358e-03, -7.49565090e-04,  4.87368300e-04,
       -1.21298939e-03,  8.97024736e-04,  1.36965895e-03,  9.44704713e-04,
       -1.67132536e-04,  5.44975876e-03,  1.58652230e-03,  6.64006721e-04,
        1.91913786e-03, -1.19126144e-03, -3.26819639e-03, -1.23511056e-03,
        7.08990990e-05, -1.67132536e-04,  9.26298478e-04, -1.12948586e-03,
       -2.10643042e-03,  1.73707986e-03, -1.67132536e-04,  1.85752357e-03,
       -9.60331465e-05])

In [84]:

filt = gs.best_estimator_.named_steps['model'].coef_ > 1e-2

In [87]:

gs.best_estimator_.named_steps['model'].coef_

array([ 0.        , -0.04078219,  0.03081593, -0.        , -0.        ,
        0.        , -0.        ,  0.        , -0.        , -0.        ,
        0.00079319, -0.        ,  0.        ,  0.        ,  0.        ,
       -0.        , -0.        , -0.        , -0.00523065, -0.        ,
        0.        ,  0.03756816,  0.        ,  0.        ,  0.        ,
        0.        ,  0.01959081,  0.        ,  0.        , -0.01059117,
        0.03595327, -0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        ,  0.        , -0.        ,  0.        ,
        0.        ,  0.        , -0.        , -0.        ,  0.        ,
        0.        , -0.        , -0.        ,  0.        , -0.        ,
        0.        , -0.        ,  0.        ,  0.        , -0.        ,
       -0.        ,  0.        , -0.        , -0.        , -0.        ,
        0.        ,  0.        ,  0.        , -0.        , -0.        ,
       -0.        , -0.        , -0.        , -0.        , -0.  

In [91]:
gs.best_estimator_.named_steps['model'].intercept_

-0.00016713253627155132

In [92]:

gs.best_params_

{'model__alpha': 8.641139163738937e-06,
 'model__fit_intercept': True,
 'model__normalize': False}